In [13]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from rosa import ROSA, RobotSystemPrompts

import subprocess
from typing import Tuple
# Load environment variables
load_dotenv()

# Initialize OpenAI LLM using langchain_openai
openai_llm = ChatOpenAI(
    model_name="gpt-4o",  # Replace with your desired model
    temperature=0,        # Adjust as needed
    max_tokens=None,      # Set maximum token count (None for default)
    timeout=5,         # Optional timeout in seconds
    max_retries=2,        # Number of retries for API calls
    openai_api_key=os.getenv("OPENAI_API_KEY")  # API key from .env
)



# @tool 
# def go_to_goal(x: float, y: float, theta: float):
#     """
#     Move the robot to a specific point in the world with an x,y coordinate and angle theta. x and y are in meters and theta is in radians. Be sure to make conversions to the correct unit where necessary.
#     """
#     # Implement the logic to move the robot to the specified point
#     print(f"Moving robot to point: ({x}, {y}, {theta})")

def execute_ros_command(command: str) -> Tuple[bool, str]:
    """
    Execute a ROS2 command.

    :param command: The ROS2 command to execute.
    :return: A tuple containing a boolean indicating success and the output of the command.
    """

    # Validate the command is a proper ROS2 command
    cmd = command.split(" ")
    valid_ros2_commands = ["node", "topic", "service", "param", "doctor"]

    if len(cmd) < 2:
        raise ValueError(f"'{command}' is not a valid ROS2 command.")
    if cmd[0] != "ros2":
        raise ValueError(f"'{command}' is not a valid ROS2 command.")
    if cmd[1] not in valid_ros2_commands:
        raise ValueError(f"'ros2 {cmd[1]}' is not a valid ros2 subcommand.")

    try:
        output = subprocess.check_output(command, shell=True).decode()
        return True, output
    except Exception as e:
        return False, str(e)


@tool
def go_to_goal(
    x: float,
    y: float,
    theta: float
):
    """
    Move the robot to a specific point in the world with an x,y coordinate and angle theta.
    Check if the point is occupied before sending the goal

    :param x: The x coordinate of the point to move to in meters.
    :param y: The y coordinate of the point to move to in meters.
    :param theta: The angle to move to in radians.
    """

    cmd = f"ros2 topic pub /nav2_goal_pose std_msgs/Float64MultiArray \"data: [{x}, {y}, {theta}]\" --once"
    success,output = execute_ros_command(cmd)

    if success:
        print(f"Successfully sent command to move to ({x}, {y}, {theta})")
    else:
        print(f"Failed to send command ({x}, {y}, {theta}): {output}")
    
    
    # Implement the logic to move the robot to the specified point
    print(f"Moving robot to point: ({x}, {y}, {theta})")
    
@tool
def check_map_occupancy(x: float, y: float) -> bool:
    """
    Check if a specific point in the map is occupied. Called when a user wants to move the robot to a point. Cannot go to a point that is occupied.

    :output is a boolean true or fale. true means the coordinate is occupied. false means it isnt
    """

    cmd = f"ros2 service call /check_coordinate a24_interfaces/srv/CheckCoordinates \"{{x: {x}, y: {y}}}\""
    print(cmd)
    success, output = execute_ros_command(cmd)
  
    print(f"Checking map occupancy for point: ({x}, {y})")
    print(f"Output: {output}")

    is_occupied_val = "is_occupied=True" in output 
    return is_occupied_val #tells us if the map is occuped

@tool
def take_image():
    """
    Takes an image of what the robot is seeing in its camera and saves it as a .jpg image to this directory of the same "image.jpg"
    """
    
prompts = RobotSystemPrompts(
    embodiment_and_persona="You are are a 2 wheeled robot that can move around the house with the use of SLAM using a 2d lidar. you can see the world around you and you can move around the house. you can also navigate to a specific point in the house.",
    critical_instructions="Be sure to make conversions to the correct unit where necessary. Do not go to a point that is occupied in the map.",
    about_your_capabilities="You can move to a point in the world with an x,y coordinate and angle theta. You have a camera so you can take pictures and infer from the images. You also have a depth camera so you can find the distance to objects in the world.",
    mission_and_objectives="your mission is to at like a pet that follows instructions from the home owners"
)

# Pass the LLM to ROSA
rosa_instance = ROSA(
    ros_version=2,  # Specify your ROS version (1 or 2)
    llm=openai_llm,
    tools=[go_to_goal, check_map_occupancy],
    prompts=prompts
)

# Use ROSA
# result = rosa_instance.invoke("check if the point 1,1 is occupied")
result = rosa_instance.invoke("move to point 1,1")
print(result)


ros2 service call /check_coordinate a24_interfaces/srv/CheckCoordinates "{x: 1.0, y: 1.0}"
Checking map occupancy for point: (1.0, 1.0)
Output: requester: making request: a24_interfaces.srv.CheckCoordinates_Request(x=1.0, y=1.0)

response:
a24_interfaces.srv.CheckCoordinates_Response(success=True, is_occupied=False)


Successfully sent command to move to (1.0, 1.0, 0.0)
Moving robot to point: (1.0, 1.0, 0.0)
I have successfully moved to the point (1, 1). If you have any other instructions, feel free to let me know!
